In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.ensemble import GradientBoostingRegressor


In [2]:
house = pd.read_csv('./datasets/train_new.csv')
house_test=pd.read_csv('./datasets/test.csv')

C:\Users\wongh\AppData\Local\Temp\ipykernel_24256\3092648055.py:2: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  house_test=pd.read_csv('./datasets/test.csv')


In [3]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])

    # Compute the Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Raffles Place coordinates
raffles_place_latitude, raffles_place_longitude = 1.2833, 103.8501

# Calculate the distance between each location and Raffles Place
house_test['Distance_from_CBD'] = haversine_distance(house_test['Latitude'], house_test['Longitude'],
                                                       raffles_place_latitude, raffles_place_longitude)

In [4]:
house_test['age_at_tranc']=house_test['Tranc_Year']-house_test['lease_commence_date']

In [5]:
house['flat_type'].sort_values().unique()

array(['1 ROOM', '2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE',
       'MULTI-GENERATION'], dtype=object)

In [6]:
rooms={}
for room in house['flat_type'].sort_values().unique():
    house_room=house[house['flat_type']==room].drop(['flat_type'],axis=1)
    rooms.update({room:house_room})

In [7]:
rooms['2 ROOM']= rooms['2 ROOM'][rooms['2 ROOM']['flat_model'] != 'DBSS']
rooms['2 ROOM'][rooms['2 ROOM']['flat_model'] == 'DBSS']

,floor_area_sqm,flat_model,resale_price,town,Tranc_Year,mid,mrt_nearest_distance,dist_from_city,age_at_tranc


In [8]:
pipelines={}
actual=[]
predicted=[]

# Fitting the model for different room types
for room_type in list(rooms):
   
    # Extract the data for each flat_type
    data = rooms[room_type]
    X = data.drop('resale_price',axis=1)
    y = data['resale_price']
    
    # Setting the train test split data
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
    
    # Append y_test into actual list
    actual.append(y_test)
    
    # Creating the numerical and categorical columns
    cat_cols = [cname for cname in X.columns if X[cname].dtype == 'object']
    num_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
                  
    # set up the column transformer to apply OneHotEncoder and StandardScaler
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), cat_cols),
            ('knn',KNNImputer(n_neighbors=30),num_cols), ('num', StandardScaler(), num_cols)
        ])
    
    # set up the pipeline for 1 Room with column transformer and GB regression model
    pipeline_1rm = Pipeline([
        ('preprocessor', preprocessor),
        ('model', GradientBoostingRegressor(loss='absolute_error', learning_rate=0.1, random_state=42))])
    
    # setup the pipeline for MultiGeneration with column transformer and RandomForest regression model
    pipeline_multi= Pipeline([
        ('preprocessor', preprocessor),
        ('model', GradientBoostingRegressor(loss='absolute_error', learning_rate=0.1, random_state=42))])
    
    # set up the pipeline for other flat types with column transformer and GB regression model
    pipeline_rest = Pipeline([
        ('preprocessor', preprocessor),
        ('model', XGBRegressor(n_estimators=500, learning_rate=0.05, n_jobs=-1))])
    
    # Fitting the model for 1 Room
    if room_type=='1 ROOM':
        pipe=pipeline_1rm.fit(X_train, y_train)
        pipelines.update({room_type:pipe})
        pipeline=pipeline_1rm
    # Fitting the model for other flat types
    elif room_type=='MULTI-GENERATION':
        pipe=pipeline_multi.fit(X_train, y_train)
        pipelines.update({room_type:pipe})
        pipeline=pipeline_multi
    
        
    else:
        pipe=pipeline_rest.fit(X_train, y_train)
        pipelines.update({room_type:pipe})
        pipeline=pipeline_rest
        
    # Checking the predicted y values
    y_pred = pipeline.predict(X_test)
    
    # Append y_pred into predicted list
    predicted.append(y_pred)
    
    # Get MSE, RMSE, MAE R^2 scores
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    score = pipeline.score(X_test, y_test)
    
    # Print the scores
    print(f'For {room_type},' "\n")
    print("R^2:", score, "\n")
    print("MSE: ", mse, "\n")
    print("RMSE: ", rmse, "\n")
    print("MAE: ", mae, "\n")


For 1 ROOM,

R^2: 0.8323261267816742 

MSE:  94025010.01301686 

RMSE:  9696.649421991953 

MAE:  8134.829774066344 

For 2 ROOM,

R^2: 0.8311618851892227 

MSE:  201773257.51696873 

RMSE:  14204.691391120356 

MAE:  10918.68552110818 

For 3 ROOM,

R^2: 0.9211329694402107 

MSE:  356803442.1143368 

RMSE:  18889.241438298595 

MAE:  14020.61375328021 

For 4 ROOM,

R^2: 0.9551482372209725 

MSE:  550384814.4599298 

RMSE:  23460.281636415402 

MAE:  17267.834159204285 

For 5 ROOM,

R^2: 0.9531824206720587 

MSE:  807132321.5214297 

RMSE:  28410.07429630253 

MAE:  21220.19377814774 

For EXECUTIVE,

R^2: 0.909629712702493 

MSE:  1081572224.7468567 

RMSE:  32887.26538870109 

MAE:  24439.660029190993 

For MULTI-GENERATION,

R^2: 0.7340683471781417 

MSE:  1464025982.4399796 

RMSE:  38262.59246888506 

MAE:  30277.152592020877 



In [9]:
pipelines

{'1 ROOM': Pipeline(steps=[('preprocessor',
                  ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                   ['flat_model', 'town']),
                                                  ('knn',
                                                   KNNImputer(n_neighbors=30),
                                                   ['floor_area_sqm',
                                                    'Tranc_Year', 'mid',
                                                    'mrt_nearest_distance',
                                                    'dist_from_city',
                                                    'age_at_tranc']),
                                                  ('num', StandardScaler(),
                                                   ['floor_area_sqm',
                                                    'Tranc_Year', 'mid',
                                                    'mrt_nearest_distance',
              

In [10]:
actuals=np.concatenate(actual)

In [11]:
predicts=np.concatenate(predicted)

In [12]:
# Create a compare dataframe of actual and predicted results
compare=pd.DataFrame({'Actual':actuals,'Predicted':predicts})

# Apply Function to round off to nearest 1000
def round_to_nearest_thousands(x):
    return round(x / 1000) * 1000

compare['Predicted'] = compare['Predicted'].apply(lambda x:round_to_nearest_thousands(x))

# Get Dataframe to show the price diff between Actual and Predicted
compare['Price Diff'] = compare['Predicted'] - compare['Actual']
compare['Price Diff Percent'] = (compare['Predicted'] - compare['Actual']) / compare['Actual'] *100
compare.head(100)


,Actual,Predicted,Price Diff,Price Diff Percent
0,180000.0,190000,10000.0,5.555556
1,213000.0,201000,-12000.0,-5.633803
2,190000.0,201000,11000.0,5.789474
3,235000.0,230000,-5000.0,-2.127660
4,210000.0,201000,-9000.0,-4.285714
...,...,...,...,...
95,255000.0,245000,-10000.0,-3.921569
96,257000.0,244000,-13000.0,-5.058366
97,260000.0,261000,1000.0,0.384615
98,250000.0,253000,3000.0,1.200000


In [13]:
compare.shape

(30129, 4)

In [14]:
np.abs(-10)

10

In [15]:
# Checking how many price difference percentage above 10%
compare.loc[(compare['Price Diff Percent']<=10.0)&(compare['Price Diff Percent']>=-10)].value_counts().sum()

28282

Total number of price difference percentage <= 10% is 2216 which is roughly 93.8% of the whole test set.

In [21]:
# Calculate the distance between each location and Raffles Place for the df_test
house_test['dist_from_city'] = haversine_distance(house_test['Latitude'], house_test['Longitude'],
                                                       raffles_place_latitude, raffles_place_longitude)

In [22]:
# Initialize an empty list to store the data
data = {'Id':[],'Predicted':[]}

# Loop through a range of ids
for id in list(house_test['id']):
    
    # Calculate the predicted price based on the id 
    # Get the sample data for the given id
    sample_data = house_test[house_test['id'] == id]
    
    # Get the flat type for the sample data
    flat_type = sample_data['flat_type'].iloc[0]
    
    # Get the pipeline associated with the flat type
    pipeline = pipelines[flat_type]
    
    # Drop the ID and flat type columns as they are not required for prediction
    sample_data_pred = sample_data.drop(['id', 'flat_type'], axis=1)
    
    # Predict the resale price using the pipeline
    predicted_price = pipeline.predict(sample_data_pred)
        
    # Append the id and predicted price to the data list
    data['Id'].append(id)
    data['Predicted'].append(predicted_price[0])

In [23]:
data

{'Id': [114982,
  95653,
  40303,
  109506,
  100149,
  7610,
  61101,
  68167,
  65701,
  56039,
  81919,
  99768,
  124053,
  3942,
  118904,
  30060,
  185729,
  46357,
  20662,
  102274,
  53369,
  9719,
  20145,
  159051,
  31481,
  142247,
  29119,
  151448,
  60594,
  83400,
  40168,
  52407,
  127232,
  131188,
  952,
  166230,
  144844,
  33207,
  28355,
  179918,
  162451,
  19390,
  114861,
  132259,
  180033,
  97052,
  94224,
  32295,
  171908,
  80236,
  156193,
  54494,
  174530,
  143835,
  26275,
  61627,
  49813,
  3466,
  117691,
  132738,
  114211,
  116741,
  39246,
  150011,
  182319,
  32441,
  18842,
  99428,
  96435,
  134803,
  171636,
  115944,
  149001,
  19084,
  178128,
  74818,
  49603,
  86262,
  14520,
  57209,
  148575,
  33,
  180401,
  87114,
  16450,
  28848,
  11519,
  7202,
  2843,
  156073,
  132217,
  140349,
  34614,
  135594,
  170933,
  62586,
  145868,
  15155,
  61290,
  184569,
  169996,
  7026,
  128356,
  134501,
  163406,
  56796,
  174

In [24]:
sub_reg = pd.DataFrame(data)

In [25]:
sub_reg.to_csv('./datasets/sub_regGBR.csv', index=False)